In [1]:
import json

from urllib.request import urlopen

from datetime import datetime

# To help see if we have existing data or not.
import os

# For Rick
import pickle

In [2]:
testing = False

with open('../data/raw/all_users', 'rb+') as file :
    all_users = pickle.load(file)
with open('../data/raw/mapped_users', 'rb+') as file :
    mapped_users = pickle.load(file)
    mapped_users = set(mapped_users)
with open('../data/raw/skipped_users', 'rb+') as file :
    skipped_users = pickle.load(file)
    skipped_users = set(skipped_users)
with open('../data/raw/relationship_scraper_counter', 'rb+') as file :
    counter = pickle.load(file)

# Since the "all_relationships" file gets honkin big, and since all
# the info we really need to operate the loop is already in the other
# lists, we don't keep the whole file in memory; we just add to a
# dummy list and then + that list to the existing list while saving.
relationship_holder = []

In [3]:
users_to_map = [user for user in all_users if user not in mapped_users]
users_to_map = [user for user in users_to_map if user not in skipped_users]

In [ ]:
remembered_len = len(all_users)
print(f'Starting scraping at {remembered_len} known users')

try :
    while True :
        if testing : print(f'1 Finding user')
        user = users_to_map[counter]
        if testing : print(f'1 User found')
        try :
            if testing : print(f'2 Getting response')
            response = urlopen(f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key=83A112A706DF72CB0920A20F28F04252&steamid={user}&relationship=friend")
            if testing : print(f'2 Response gotted')
            response = json.loads(response.read())
            relationships = response['friendslist']['friends']
            for relationship in relationships :
                if testing : print(f'3 Checking all users')
                if relationship['steamid'] not in all_users :
                    all_users.append(relationship['steamid'])
                if testing : print(f'3 Checked')
                if testing : print(f'4 Checking mapped users')
                if relationship['steamid'] not in mapped_users :
                    if testing : print(f'4 Checked')
                    if testing : print(f'5 Making and adding relationship')
                    new_relationship = tuple([user, relationship['steamid'], relationship['friend_since']])
                    relationship_holder.append(new_relationship)
                    if testing : print(f'5 Relationship done')
            if testing : print(f'6 Adding mapped users')
            mapped_users.add(user)
            if testing : print(f'Processed: {counter+1}   Known: {len(all_users)}')
        except Exception as e :
            if testing : 
                print(Exception)
                print(f'7 Adding skipped user')
                print(f'Private user')
            skipped_users.add(user)
            if testing : print(f'7 Skipped user added')
        
        counter += 1

        if counter % 100 == 0 :
            print(f'8 Saving')
            with open('../data/raw/all_users.pkl', 'wb+') as file :
                pickle.dump(all_users, file)
            with open('../data/raw/mapped_users.pkl', 'wb+') as file :
                pickle.dump(mapped_users, file)
            with open('../data/raw/skipped_users.pkl', 'wb+') as file :
                pickle.dump(skipped_users, file)
            with open('../data/raw/relationship_scraper_counter.pkl', 'wb+') as file :
                pickle.dump(counter, file)
            with open('../data/raw/all_relationships.pkl', 'rb+') as file :
                existing_relationships = pickle.load(file)
                all_relationships = existing_relationships + relationship_holder
                pickle.dump(all_relationships, file)
            print(f'Users added: {len(all_users) - remembered_len}')
            remembered_len = len(all_users)
            print(datetime.now())

except :
    print('Something fatal happened!')
    with open('../data/raw/CRASH_all_users.pkl', 'wb+') as file :
        pickle.dump(all_users, file)
    with open('../data/raw/CRASH_mapped_users.pkl', 'wb+') as file :
        pickle.dump(mapped_users, file)
    with open('../data/raw/CRASH_skipped_users.pkl', 'wb+') as file :
        pickle.dump(skipped_users, file)
    with open('../data/raw/CRASH_all_relationships.pkl', 'wb+') as file :
        existing_relationships = pickle.load(file)
        all_relationships = existing_relationships + relationship_holder
        pickle.dump(all_relationships, file)